In [10]:
import pandas as pd
import re
import numpy as np

In [11]:
config_algo_names = ['CommonChars', 'IsPalindrome', 'ReverseString','ReverseArray', 'BinarySearchStrings',
                     'Multiples','Power', 'GetMiddle',  'InsertionSort','SquareRoot']

config_unique_code_variable = "PD16_01"

config_answer_variables = ['PO03_01', 'PO08_01', 'PO13_01', 'PO18_01', 'PO23_01',
                           'PO28_01', 'PO33_01', 'PO38_01', 'PO43_01', 'PO48_01']

config_difficulty_variables = ['PO04', 'PO09', 'PO14', 'PO19', 'PO24',
                               'PO30', 'PO34', 'PO39', 'PO44', 'PO49']

config_time_variables = ['TIME002','TIME004','TIME006','TIME008', 'TIME010',
                         'TIME012','TIME014','TIME016','TIME018', 'TIME020']

config_click_variables = ['PO01_01', 'PO06_01', 'PO11_01', 'PO16_01', 'PO21_01',
                            'PO26_01', 'PO31_01', 'PO36_01', 'PO41_01', 'PO46_01']

config_correctness_variables = ['^5$', '^(?i)true$', '^olleH$', '^13294$', '^2$', 
                                '^32$', '^16$', '^rm$', '^13479$', '^1357$']

config_datasheet = r'./data/raw/posttest.xlsx'

In [12]:
raw = pd.read_excel(config_datasheet)
df = pd.DataFrame(raw)
df = df.dropna()

from scipy.stats import ttest_ind


In [13]:
df_matrix = []
raw = pd.read_excel(config_datasheet)

# load data
for row, _stuff in enumerate(config_algo_names):
    df_array = []
    df = pd.DataFrame(raw, columns = [config_unique_code_variable,
                                      config_answer_variables[row],
                                      config_correctness_variables[row],
                                      config_difficulty_variables[row],
                                      config_time_variables[row],
                                      config_click_variables[row],])
    #df = df.dropna()
    df_array.append(df)
    df_matrix.append(df_array)

In [14]:
cols = ["unique_code", "response_time", "correctness", "difficulty", "flag", "algo_name", "click_data"]

df_tmp = pd.DataFrame(columns=cols)

for row_idx, df_row in enumerate(df_matrix):
    algo_name = config_algo_names[row_idx]
    for col, df in enumerate(df_row): 
        for index, row in df.iterrows():
            unique_code = row["PD16_01"]
            response_time = int(row[config_time_variables[row_idx]])
            answer = row[config_answer_variables[row_idx]]
            difficulty = row[config_difficulty_variables[row_idx]]
            click_data = row[config_click_variables[row_idx]]
            
            if row_idx == 3:
                answer = str(row[config_answer_variables[row_idx]])
                rep = {"{": "", "+": "", " ":"", ",":"", "[":"", "]":"", "}":""} # define desired replacements here
                rep = dict((re.escape(k), v) for k, v in rep.items()) 
                pattern = re.compile("|".join(rep.keys()))
                answer = pattern.sub(lambda m: rep[re.escape(m.group(0))], answer)
            elif row_idx == 8:
                answer = str(row[config_answer_variables[row_idx]])
                rep = {"{": "", "+": "", " ":"", ",":"", "[":"", "]":"", "}":""} # define desired replacements here
                rep = dict((re.escape(k), v) for k, v in rep.items()) 
                pattern = re.compile("|".join(rep.keys()))
                answer = pattern.sub(lambda m: rep[re.escape(m.group(0))], answer)
            elif row_idx == 9:
                answer = str(row[config_answer_variables[row_idx]])
                rep = {".0": "", " ":"", ",":""} # define desired replacements here
                rep = dict((re.escape(k), v) for k, v in rep.items()) 
                pattern = re.compile("|".join(rep.keys()))
                answer = pattern.sub(lambda m: rep[re.escape(m.group(0))], answer)
            else:
                answer = row[config_answer_variables[row_idx]]
                
            correctness = bool(re.match(config_correctness_variables[row_idx], str(answer)))

            flag = ""
            data = pd.DataFrame([[unique_code, response_time, correctness, difficulty, flag, algo_name, click_data]], columns=cols)
                  
            df_tmp = df_tmp.append(data)


In [15]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
cols = ["Unique Code", "ResponseTime", "Algorithm", "Flag", "Correctness", "Difficulty", "ClickData"]

def fill(list_data, dataframe):
    for idx, row in dataframe.iterrows():
        response_time = row["response_time"]
        algorithm = row["algo_name"]
        flag = row["flag"]
        click_data = row["click_data"]
        correctness = row["correctness"]
        difficulty = row["difficulty"]
        unique_code = row["unique_code"]
        list_data.append([unique_code, response_time, algorithm, flag, correctness, difficulty, click_data])

data = []
fill(data, df_tmp)
df = pd.DataFrame(data, columns=cols)
df.to_excel("./results/preprocessed/preprocessed_posttest.xlsx", index=False)